In [1]:
import pandas as pd, numpy as np, os, seaborn as sns, librosa
import matplotlib.pyplot as plt

In [2]:
import torchaudio


ModuleNotFoundError: No module named 'torchaudio'

### Dataset-> Toronto Emotional Speech dataset

In [ ]:
paths=[]
labels=[]
for dirname, _, filenames in os.walk('/path'):
    for filename in filenames:
        paths.appned(os.path.join(dirname,filename))
        label=filename.split('_')[-1]
        label=label.split('.')[0]
        labels.append(label.lower())
    if len(paths)==2800:
        break
print('Dataset loaded')


In [ ]:
len(paths)

In [ ]:
labels[:5]

In [ ]:
df=pd.DataFrame()

In [ ]:
df['labels'].value_counts()

## Exploratory Data Analysis

In [ ]:
sns.countplot(data=df,x='labels')

In [4]:
def waveplot(data,sr,emotion):
    plt.figure(figsize=(10,4))
    plt.title(emotion,size=20)
    librosa.display.waveshow(data,sr=sr)
    plt.show()

def spectrogram(data,sr,emotion):
    x=librosa.stft(data)
    xdb=librosa.amplitude_to_db(abs(x))
    plt.figure(figsize=(11,4))
    plt.title(emotion,size=20)
    librosa.display.specshow(xdb,sr=sr,x_axis="time",y_axis="hz")
    plt.colorbar()

In [ ]:
emotion="fear"
path=np.array()